# Emerging Technologies

## Problem 1: Generating Random Boolean Functions

### Problem Overview

The Deutsch–Jozsa algorithm is one of the earliest quantum algorithms to demonstrate a clear separation between classical and quantum computation. It determines whether a Boolean function is **constant** or **balanced** using fewer evaluations than would be required classically.

In this problem, we simulate the type of functions used by the Deutsch–Jozsa algorithm. Specifically, we create a Python function that randomly returns a Boolean function with **four Boolean inputs** and **one Boolean output**, where the function is guaranteed to be either a

**Constant** — returns the same value (always `True` or always `False`) for all inputs, or  
**Balanced** — returns `True` for exactly half of the possible inputs and `False` for the other half.

This models the promise problem used in the Deutsch–Jozsa algorithm ([IBM Quantum Learning – Deutsch–Jozsa Algorithm](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa)).

### Background

A Boolean function with four inputs takes values from:

\[
f: \{0,1\}^4 \rightarrow \{0,1\}
\]

There are \(2^4 = 16\) possible input combinations therefore:

A **constant function** returns the same output for all 16 inputs.
A **balanced function** returns `True` for exactly 8 inputs and `False` for the other 8.

In the worse cases determining whether such functions as stated previously (constant or balanced) 
requires checking more then half of the inputs, however, the Deutsh-Josza quantam algorithm can 
determine this with just a single evaluation using quantum parallelism and interference ([Nielsen & Chuang, *Quantum Computation and Quantum Information*]).

This problem focuses on constructing suitable test functions in a classical environment to better understand the structure of the problem the quantum algorithm solves.

### Approach

To generate a random function that meets these requirements:

1. **Enumerate all possible inputs**  
   There are 16 possible combinations of four Boolean values.

2. **Randomly choose function type**
   With equal probability, select either **constant** or **balanced**.

3. **If constant**
    Randomly choose `True` or `False`.
    Return a function that always produces this value.

4. **If balanced**
    Randomly select 8 of the 16 inputs to map to `True`.
    The remaining inputs map to `False`.

5. **Return the generated function**
   The function itself will accept four Boolean arguments and return a Boolean result.

This IS how oracle functions are treated in theoretical descriptions of the Deutsch–Jozsa algorithm.


### Explanation of Key Ideas

I use Python’s `itertools.product` to generate all possible input combinations ([Python documentation](https://docs.python.org/3/library/itertools.html#itertools.product)).
A dictionary is used to store the mapping from input tuples to output values.
A nested function (closure) is returned so the mapping is preserved without exposing internal implementation details.

This approach reflects how black-box oracle functions are conceptualised in quantum algorithms: the internal logic is hidden, and only input-output behaviour is observable.



In [2]:
# itertools.product generates all possible combinations of Boolean inputs
# Documentation: https://docs.python.org/3/library/itertools.html#itertools.product
from itertools import product

# random is used to randomly choose function type and outputs
# Documentation: https://docs.python.org/3/library/random.html
import random


A Boolean function with four inputs has 
2^4 = 16 possible input combinations.
We use `itertools.product` to systematically generate all tuples of four Boolean values.

In [3]:
# Generate all possible combinations of four Boolean inputs
inputs = list(product([False, True], repeat=4))

# Show number of combinations
len(inputs)


16

The function `random_constant_balanced` will:

Randomly decide whether to create a constant or balanced function

Create an internal mapping of inputs to outputs

Return a function that looks up outputs from this mapping

In [4]:
def random_constant_balanced():
    """Return a random Boolean function of four inputs that is either constant or balanced."""
    
    # All possible inputs
    inputs = list(product([False, True], repeat=4))
    
    # Randomly choose function type
    function_type = random.choice(["constant", "balanced"])
    
    # Dictionary to store input-output mapping
    mapping = {}
    
    if function_type == "constant":
        # Choose a single Boolean output
        output_value = random.choice([True, False])
        # Assign same output to all inputs
        for inp in inputs:
            mapping[inp] = output_value
            
    else:  # balanced
        # Randomly select half of the inputs to map to True
        true_inputs = set(random.sample(inputs, len(inputs)//2))
        
        for inp in inputs:
            mapping[inp] = inp in true_inputs

    # Return a function that looks up values from mapping
    def f(a, b, c, d):
        return mapping[(a, b, c, d)]
    
    return f


A dictionary is used to store the mapping between input tuples and output values.
This simulates a black box function (oracle) where you can query outputs but do not see internal logic much similar to how functions are treated in quantum algorithms.

In [5]:
# Generate a random function
f = random_constant_balanced()

# Test it on a few inputs
print(f(False, False, False, False))
print(f(True, True, True, True))
print(f(True, False, True, False))


False
True
False


## Problem 2 – Classical Testing for Function Type

### Problem Overview

Deutsch’s algorithm is designed to highlight a separation between classical and quantum computation by solving a specific promise problem more efficiently on a quantum computer.

Before considering the quantum solution, it is important to understand the **classical cost** of determining whether a Boolean function is constant or balanced. In this problem I will implement a classical algorithm that analyzes a function generated in Problem 1 and determines its type with certainty.

### Background

The input function is guaranteed to be either constant or balanced. For a function with four Boolean inputs, there are 16 possible input combinations.

Classically the only way to learn about a black-box function is to **evaluate it on specific inputs**. In the worst case a classical algorithm must evaluate the function on more than half of the possible inputs to be absolutely certain that it is balanced rather than constant.

This contrasts with Deutsch’s quantum algorithm, which can determine the function type using a single evaluation by exploiting quantum superposition and interference.

### Approach

To determine whether the function is constant or balanced with certainty:

1. Evaluate the function on all possible input combinations.
2. Collect the output values.
3. If all outputs are identical, the function is constant.
4. If the outputs contain an equal number of `True` and `False` values, the function is balanced.

Because the function is guaranteed to be one of these two types, this approach will always return a correct result.
